In [1]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
# tf.enable_eager_execution()

import numpy as np
import os
import time
from IPython.display import Image

In [2]:
# Use your own file:
path_to_file = "try_with_your_own_file.txt"

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding = "ISO-8859-1")
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

NameError: name 'path_to_file' is not defined